# Part Three
## Configuring access internally / externally

We have created a container that hosts a website in the previous part.
**Let's continue from there.**
To start let's check our available images

In [2]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             baseweb             34079efab3e4        About an hour ago   373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


**And now let's have a look on all the containers that we have ran**

In [3]:
docker ps -a

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                           PORTS                  NAMES
f9fd0dbebbeb        centos6:baseweb     "/bin/bash"         58 minutes ago      Exited (255) 10 minutes ago      0.0.0.0:8080->80/tcp   webtest
695d1ddda23e        centos6:baseweb     "/bin/bash"         About an hour ago   Exited (137) About an hour ago                          fervent_wright
548f70d44cb2        centos:centos6      "/bin/bash"         2 hours ago         Exited (137) About an hour ago                          reverent_pasteur


## Setting up container
### Commit
We can still see the container we created previously. Let's save that one by committing it.

In [4]:
docker commit webtest centos6:serverv1

sha256:be824f20375e5cc3ef3e2d1ef798f93697edf4d0f59b05e55f8c14648ba6cc60


**Let's have a look**

In [5]:
docker images

REPOSITORY          TAG                 IMAGE ID            CREATED             SIZE
centos6             serverv1            be824f20375e        11 seconds ago      373 MB
centos6             baseweb             34079efab3e4        About an hour ago   373 MB
centos              centos6             8315978ceaaa        4 months ago        195 MB


**There we can see our new serverv1!**
Let's have a look on how this looks like

In [10]:
docker run --name contserv1 -dit centos6:serverv1 /bin/bash

62043409c68ded137b9eba4fd8949a28f302f292c1af731e6c116cd6381d4a12


In [11]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS                  PORTS               NAMES
62043409c68d        centos6:serverv1    "/bin/bash"         2 seconds ago       Up Less than a second   80/tcp              contserv1


In [12]:
docker logs contserv1

Starting httpd: httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.2 for ServerName
[  OK  ]
Starting sshd: [  OK  ]


**We can see that all is good so far....**
### However params are not saved!

In [14]:
docker exec contserv1 df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          63G  1.7G   58G   3% /
tmpfs          1000M     0 1000M   0% /dev
tmpfs          1000M     0 1000M   0% /sys/fs/cgroup
/dev/vda1        63G  1.7G   58G   3% /etc/resolv.conf
/dev/vda1        63G  1.7G   58G   3% /etc/hostname
/dev/vda1        63G  1.7G   58G   3% /etc/hosts
shm              64M     0   64M   0% /dev/shm
tmpfs          1000M     0 1000M   0% /proc/kcore
tmpfs          1000M     0 1000M   0% /proc/timer_list
tmpfs          1000M     0 1000M   0% /proc/sched_debug
tmpfs          1000M     0 1000M   0% /sys/firmware


**As you can notice, neither our ports, nor our volume is being saved!**
That sucks...
We didnt inherit all the changes.
### Fixing the problem
We want to:

* Mount underlying system as before
* Map port

In [15]:
docker run --name=externalweb -p 8081:80 -v $(pwd)/dockerwww:/var/www/html -dit centos6:serverv1 /bin/bash

af3ac3489de7e106375a5c227c75941e968d3bf0c36639550263c89e4edd803f


In [16]:
docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS                  NAMES
af3ac3489de7        centos6:serverv1    "/bin/bash"         7 seconds ago       Up 5 seconds        0.0.0.0:8081->80/tcp   externalweb
62043409c68d        centos6:serverv1    "/bin/bash"         4 minutes ago       Up 4 minutes        80/tcp                 contserv1


In [18]:
docker logs externalweb

Starting httpd: httpd: Could not reliably determine the server's fully qualified domain name, using 172.17.0.3 for ServerName
[  OK  ]
Starting sshd: [  OK  ]


**Right we can see that our docker instance is running, and the port mapping worked successfully**

In [17]:
ifconfig

lo0: flags=8049<UP,LOOPBACK,RUNNING,MULTICAST> mtu 16384
	options=3<RXCSUM,TXCSUM>
	inet6 ::1 prefixlen 128 
	inet 127.0.0.1 netmask 0xff000000 
	inet6 fe80::1%lo0 prefixlen 64 scopeid 0x1 
	nd6 options=1<PERFORMNUD>
gif0: flags=8010<POINTOPOINT,MULTICAST> mtu 1280
stf0: flags=0<> mtu 1280
en0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
	ether f4:5c:89:9f:ff:d9 
	inet6 fe80::f65c:89ff:fe9f:ffd9%en0 prefixlen 64 scopeid 0x4 
	inet 192.168.15.93 netmask 0xffffff00 broadcast 192.168.15.255
	nd6 options=1<PERFORMNUD>
	media: autoselect
	status: active
en1: flags=963<UP,BROADCAST,SMART,RUNNING,PROMISC,SIMPLEX> mtu 1500
	options=60<TSO4,TSO6>
	ether 6a:00:01:b4:37:50 
	media: autoselect <full-duplex>
	status: inactive
en2: flags=963<UP,BROADCAST,SMART,RUNNING,PROMISC,SIMPLEX> mtu 1500
	options=60<TSO4,TSO6>
	ether 6a:00:01:b4:37:51 
	media: autoselect <full-duplex>
	status: inactive
bridge0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
	options=63<

### Other benefits
We can now do different things with our container such as creating a new user, and connecting to ssh directly!